# QLearning and robot navigation

[Youtube lecture](https://www.youtube.com/watch?v=X9UhB953TDA)

## Constructor notes: starts at 6 mins

- num states: 10x10 grid making it 100 states
- num actions: 4 actions to us
- to init q table:  7 mins
  - one of the dimension is num_states & other is num_actions
  - 100 rows x 4 cols
- alpha: 7:20
  - learning rates
  - how much do you trust new info when updating q table
- gamma: 7:35
  - what is the value of future rewards?
  - discounted reward for future reward
  - immediate reward is full but for future rewards are discounted by gamma
  - 8:09 more rewards notes
    - until you reach goal, robot gets reward of -1 (except quick sand)
    - this is like you are walking across electrified floor with wet feet; so you wanna quickly get to the goal. This is what negative rewards motivate it to do: get to goal as fast as can
- rar: 9:50
  - random action rate
  - the thing that makes Q Learning works is early on, it should choose random action occasionally; this is what forces the learner to explore by trying different action than it tried last time. Using rar is the way to do that
  - book has this covered
  - this is number between 0 & 1 to indicate a probability that at each step it will choose a random action
- radr: 9:56
  - random action decay rate
  - this random action rate should decay over time and eventually become 0; once you have a perfect policy, you don't ever want to take a random action because it might not be optimal thing to do. 
  - so, you start with non zero value and decay it over time so that it approaches 0
  - radr describes how to decay rar each time your agent takes an step
  - see project description for equation to decay rar by radr
- dyna: 11:53
  - to describe how many dyna updates you should do
  - initially ignore this and just build qlearner; QLearner is 95 points and dyna is 5 points

## query(s_prime, r): 12:45

- primary function that implements q-learning query
- test harness calls this code repeatedly to give you new state you are in, s_prime, and r is reward for last action you took
- query should return an integer which is the action to take in this state s_prime
- steps to follow  within this function: 13:24
  - 1st: 
    - update q table because you got new information
    - you see that you are in new state and you are getting reward for last action you took 
    - create a separate local variable to remember which state you were in and what action you took (that resulted in your new state s_prime); with this tracking you have got your `experience tuple`: (s, a, s_prime, r) which is what you need to update your q table
  - 2nd: 14:32
    - Roll the dice to see if you should take random action or not (based on rar)
      - if you need to take random action:
        - update rar using radr
        - return random action
      - if not random action:
        - then you need to determine what action you should you take
        - remember: q table answers this problem for you
          - go to your q table
          - go the row s_prime
          - and look at the q values for each of the action that you can take and choose the one that has the highest value
          - return this action
  - 1 & 2 are all you need to do in query



## querysetstate: 16:25

- this is like when the very first time your code runs, or you just landed into new world, there is no way you can remember the last state (as we need to in the query function)
- so this function is just going to give you a starting state and you should save that state only! do not update anything in q table
- Just remember this state & next will be call to query state api


## Student questions: 17:29

Q1.17:45 How to choose random action with probability RAR?
=> call rand number generator to get number between 0 & 1; if what comes back from random number generator is less then rar you take random action
```
if rand.uniform(0.0, 1.0) <= rar: # going rogue
    a = rand.randint(0,3) # choose random direction
```

Q2. 19:30 In this problem we know there is 100 state & 4 actions. But what about actual trader, how do we know how many states there are?
=> A lot of success in applying ML algorithm to a problem involves mapping the problem you are trying to solve to the problem the learner can solve; the short answer is that it takes human craftmanship. [and some more discussion on it...]

Q3. 24:00 What is the q update?
=> lecture code has it


 

## Navigation problem: 25:40

- testworlds: 
  - bunch of csv files representing our grid world
  - 0-5 have specific meaning and is in project description
  - 3: goal state and objective is for the robot to navigate to this goal state
  - 5: special things called quick sand and if robot gets there it's a reward of -100
- sample run at 29:45


## Transforming any problem such that q learner can solve it: 32:35

Here is what you have to do:

- States: define what are the states
- Actions
- Reward (R)
- Transition matrix (T)

Learner does not know R & T and the magic of Q learning is that it can still solve it

## testqlearner.py: 34:24

- walk through

## Dyna: 40:00

- Real experience is expensive in terms of time. 
- It lets you take advantage of experience that you already have from the world and refine your q table for each real experience in the world that you take in the world. 
- it lets you converge your q table many many more times so that the number of experiences you have to have with real world is greatly reduced. 
- In fact when you run your solution with Dyna, it converges to often better solution after only few iteration then compared to the non Dyna version.
- Dyna is important when interaction with world is expensive.
  
## How dyna works: 43:30

- Remember we don't have T & R (model free learning)
- What we can do is, as the agent interacts with world build a model of T and R
- How to build model of T: (44:10)
  - T is table of 3 dim (or 3 params): S, a, S'
  - T is just the probability that you end up in S' state given you were in S and took action a
  - (44:40) to build T:
    - each time you are in state S and you take action a, observe what new state you end up in and you just count for each possible subsequent state you are in and define the probability
  - (45:05) to build R:
    - you know state you were in and what action you took and what reward you got. you can build a look up table for R(s, a)

## How to use T & R once you have it - heart of Dyna: 45:42

- when query function gets called, do everything you did before dyna, but before you pick action (either by rolling dice or other way) here what you need to do for dyna:
  - hallucinate an experience: how? 46:25
    - choose a random s and a and then look at your probability of T, which is probability of subsequent state you will end up in; consult it and pick the subsequent state according to that probability. Now, we have s, a, s'; we can also check the relevant reward from R table by looking up for R(s,a) which gives us our hallucinated r.
    - now we have hallucinated an experience but it is based on our past experience. 
    - we take this hallucinated experience and do regular q table update
  - hallucinate this to the number of times specified by dyna parameter and with each hallucination experience update the q table; with this your q table converges to the right solution much more quickly in terms of interaction with world.
  - Only catch is that the run time of your program is going to be much more longer but doesn't mean dyna is bad idea. 
- 49:55: intent of Dyna is to sample from experiences you already have to update your q table. that doesn't require you to build T or an R. There are other ways to do it w/o building T or R